In [16]:
from dotenv import load_dotenv
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.utilities import SQLDatabase
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
import streamlit as st

from langchain_community.utilities import SQLDatabase


sqlite_uri = 'sqlite:///./SK.db' 
db = SQLDatabase.from_uri(sqlite_uri)



    

In [10]:
def get_sql_chain(db):
  template = """
    You are a HR at a company. You are interacting with a employee who is asking you questions about the employees data.
    Based on the table schema below, write a SQL query that would answer the user's question. Take the conversation history into account.
    
    <SCHEMA>{schema}</SCHEMA>
    
    Conversation History: {chat_history}
    
    Write only the SQL query and nothing else. Do not wrap the SQL query in any other text, not even backticks.
    
    For example:
    Question: which 3 artists have the most tracks?
    SQL Query: SELECT ArtistId, COUNT(*) as track_count FROM Track GROUP BY ArtistId ORDER BY track_count DESC LIMIT 3;
    Question: Name 10 artists
    SQL Query: SELECT Name FROM Artist LIMIT 10;
    
    Your turn:
    
    Question: {question}
    SQL Query:
    """
    
  prompt = ChatPromptTemplate.from_template(template)
  
  llm = ChatOpenAI(model="gpt-4-0125-preview")
  
  def get_schema(_):
    return db.get_table_info()
  
  return (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | llm
    | StrOutputParser()
  )

In [11]:
def get_response(user_query: str, db: SQLDatabase, chat_history: list):
  sql_chain = get_sql_chain(db)
  
  template = """
    You are a HR at a company. You are interacting with a employee who is asking you questions about the employees data.
    Based on the table schema below, question, sql query, and sql response, write a natural language response.
    <SCHEMA>{schema}</SCHEMA>

    Conversation History: {chat_history}
    SQL Query: <SQL>{query}</SQL>
    User question: {question}
    SQL Response: {response}"""
  
  prompt = ChatPromptTemplate.from_template(template)
  
  llm = ChatOpenAI(model="gpt-4-0125-preview")
  
  chain = (
    RunnablePassthrough.assign(query=sql_chain).assign(
      schema=lambda _: db.get_table_info(),
      response=lambda vars: db.run(vars["query"]),
    )
    | prompt
    | llm
    | StrOutputParser()
  )
  
  return chain.invoke({
    "question": user_query,
    "chat_history": chat_history,
  })

In [15]:
get_response("give me all the data of the first employee",db=db,chat_history=[])

'The first employee listed in our database is Wilson K Adinolfi. Here are the details about him:\n\n- **Employee ID:** 10026\n- **Marital Status:** Single\n- **Gender:** Male\n- **Employment Status ID:** 1\n- **Department ID:** 5\n- **Performance Score ID:** 4\n- **From Diversity Job Fair ID:** 0\n- **Salary:** $62,506\n- **Termination Status:** Currently Active (Not Terminated)\n- **Position ID:** 19\n- **Position:** Production Technician I\n- **State:** MA\n- **ZIP Code:** 1960\n- **Date of Birth:** July 10, 1983\n- **Citizenship:** US Citizen\n- **Ethnicity:** Not Hispanic or Latino\n- **Race:** White\n- **Date of Hire:** July 5, 2011\n- **Date of Termination:** Still Employed (No termination date)\n- **Reason for Termination:** N/A - Still Employed\n- **Employment Status:** Active\n- **Department:** Production\n- **Manager\'s Name:** Michael Albert\n- **Manager ID:** 22.0\n- **Recruitment Source:** LinkedIn\n- **Performance Score:** Exceeds\n- **Engagement Survey Score:** 4.6\n- **